In [12]:
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pydsstools.heclib.dss import HecDss
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

folder_path = "C:\DSS FILE OUTPUTS\DSS-Sensitivity"
river_stations = ["90128", "117188", "110659", "7458"]
search_word = "Sensitivity" #Searches within the Plan Name
pathname_pattern = "/*/*/*/*/*/*/"
start_date = "26AUG2023 00:00:00"
end_date = "14SEP2023 00:00:00"

df = pd.DataFrame()

def check_identical_values(df, new_series):
    for column in df.columns:
        if np.array_equal(df[column].head(100).values, new_series.head(100).values):
            return True
    return False

for file in os.listdir(folder_path):
    if file.endswith(".dss"):
        dss_file = os.path.join(folder_path, file)
        with HecDss.Open(dss_file) as fid:
            path_list = fid.getPathnameList(pathname_pattern, sort=1)
            filtered_path_list = [
                path for path in path_list
                if any(station in path for station in river_stations)
                and search_word in path
                and ("FLOW" in path or "STAGE" in path)
                and "FLOW-CUM" not in path
            ]
            for pathname in filtered_path_list:
                ts = fid.read_ts(pathname, window=(start_date, end_date), trim_missing=True)
                if ts is not None:
                    times = np.array(ts.pytimes)
                    values = ts.values
                    header = f"{file} {pathname}"
                    temp_df = pd.DataFrame({header: pd.Series(values[~ts.nodata], index=times[~ts.nodata])})
                    if not check_identical_values(df, temp_df[header]):
                        df = pd.concat([df, temp_df], axis=1)


df.to_csv(os.path.join(folder_path, "output_temp.csv"), index_label="Date")

output_csv_path = os.path.join(folder_path, "output_temp.csv")
df_output = pd.read_csv(output_csv_path, index_col='Date', parse_dates=True)
output_dates = pd.date_range(df_output.index[0], periods=8)
new_index = output_dates.union(df_output.index[1:])


output_dates = pd.date_range(df_output.index[0], periods=7)
new_index = output_dates.union(df_output.index)

def insert_extra_rows(series):
    split_label = series.name.split('/')
    extra_rows = pd.Series(split_label[:6], index=pd.date_range(series.index[0], periods=7)[:-1])
    return pd.concat([extra_rows, series]).rename(series.name)


df_output = df_output.apply(insert_extra_rows)
df_output.to_csv(os.path.join(folder_path, "Filtered_DSS_Results.csv"), index_label="Date", na_rep="-")



grouped_data = {}
for column in df.columns:
    split_label = column.split('/')
    station = split_label[2]
    series_type = "STAGE" if "STAGE" in split_label[3] else "FLOW"
    key = (station, series_type)
    if key not in grouped_data:
        grouped_data[key] = []
    grouped_data[key].append(df[column])

from bokeh.palettes import Category20
from bokeh.models import HoverTool, Legend, LegendItem
from bokeh.plotting import figure, show, output_file, save
from bokeh.layouts import gridplot
from bokeh.io import output_notebook
from bokeh.models.sources import ColumnDataSource

output_notebook()

def plot_grouped_data_bokeh(grouped_data, folder_path):
    plots = []

    for key in grouped_data:
        p = figure(width=800, height=700, x_axis_type="datetime", title=f"{key[0]} - {key[1]}")
        legend_items = []

        for idx, series in enumerate(grouped_data[key]):
            plan_name = series.name.split("/")[-2]  # Corrected plan name extraction
            source = ColumnDataSource(data=dict(x=series.index, y=series.values, plan_name=[plan_name] * len(series)))
            color = Category20[20][idx % 20]
            line = p.line(x='x', y='y', source=source, line_width=2, color=color)
            legend_items.append(LegendItem(label=plan_name, renderers=[line]))

            hover = HoverTool(tooltips=[("Plan Name", "@plan_name"), ("Date", "@x{%F %T}"), ("Value", "@y{0.000}")], formatters={"@x": "datetime"}, renderers=[line])
            p.add_tools(hover)

        p.xaxis.axis_label = 'Date'
        unit = "CFS" if key[1] == "FLOW" else "FT"
        p.yaxis.axis_label = f'Values ({unit})'

        # Create and position the legend
        legend = Legend(items=legend_items, location="top_right")
        p.add_layout(legend)

        plots.append([p])

    output_file(os.path.join(folder_path, "Filtered_DSS_Results.html"), title="Time Series Plots by River Station and Series Type")
    layout = gridplot(plots)
    save(layout)
    # show(layout)

plot_grouped_data_bokeh(grouped_data, folder_path)





Loading BokehJS ...